In [ ]:
import os
from huggingface_hub import InferenceClient
import spacy

from google.colab import userdata

nlp = spacy.load("en_core_web_sm")


# Initialize HuggingFace API client
client = InferenceClient(
    provider="featherless-ai",
    api_key=userdata.get('HF_TOKEN'),
)

# Predefined map of adjectives to tags (90 entries)
adjective_map = {
    "Beautiful": "scenic", "Scenic": "scenic",
    "Cheap": "affordable", "Affordable": "affordable",
    "Crowded": "overpopulated", "Overpopulated": "overpopulated",
    "Expensive": "luxury", "Luxury": "luxury",
    "Quiet": "peaceful", "Peaceful": "peaceful",
    "Rustic": "rural", "Rural": "rural",
    "Romantic": "romantic", "Romantic": "romantic",
    "Clean": "hygienic", "Hygienic": "hygienic",
    "Warm": "tropical", "Tropical": "tropical",
    "Cold": "winter", "Winter": "winter",
    "Busy": "urban", "Urban": "urban",
    "Relaxing": "tranquil", "Tranquil": "tranquil",
    "Charming": "quaint", "Quaint": "quaint",
    "Historic": "cultural", "Cultural": "cultural",
    "Adventurous": "adventure", "Adventure": "adventure",
    "Safe": "secure", "Secure": "secure",
    "Family-friendly": "family", "Family": "family",
    "Friendly": "hospitable", "Hospitable": "hospitable",
    "Lively": "vibrant", "Vibrant": "vibrant",
    "Modern": "urban", "Urban": "urban",
    "Tropical": "island", "Island": "island",
    "Unique": "exotic", "Exotic": "exotic",
    "Vibrant": "lively", "Lively": "lively",
    "Isolated": "remote", "Remote": "remote",
    "Spacious": "expansive", "Expansive": "expansive",
    "Luxury": "high-end", "High-end": "high-end",
    "Breathtaking": "scenic", "Scenic": "scenic",
    "Peaceful": "calm", "Calm": "calm",
    "Exotic": "adventurous", "Adventurous": "adventurous",
    "Picturesque": "scenic", "Scenic": "scenic",
    "Majestic": "grand", "Grand": "grand",
    "Aesthetic": "artistic", "Artistic": "artistic",
    "Serene": "calm", "Calm": "calm",
    "Idyllic": "peaceful", "Peaceful": "peaceful",
    "Dynamic": "vibrant", "Vibrant": "vibrant",
    "Enchanting": "charming", "Charming": "charming",
    "Mysterious": "exotic", "Exotic": "exotic",
    "Thriving": "urban", "Urban": "urban",
    "Chilly": "cold", "Cold": "cold",
    "Diverse": "multicultural", "Multicultural": "multicultural",
    "Wilderness": "nature", "Nature": "nature",
    "Glistening": "shiny", "Shiny": "shiny",
    "Untouched": "pristine", "Pristine": "pristine",
    "Cozy": "comfortable", "Comfortable": "comfortable",
    "Lush": "green", "Green": "green",
    "Bright": "sunny", "Sunny": "sunny",
    "Vast": "expansive", "Expansive": "expansive",
    "Picturesque": "scenic", "Scenic": "scenic",
    "Delightful": "charming", "Charming": "charming",
    "Vibrant": "colorful", "Colorful": "colorful",
    "Fascinating": "intriguing", "Intriguing": "intriguing",
    "Sandy": "beach", "Beach": "beach",
    "Luminous": "bright", "Bright": "bright",
    "Breezy": "refreshing", "Refreshing": "refreshing",
    "Clear": "crystal-clear", "Crystal-clear": "crystal-clear"
}


In [ ]:
def extract_adjectives(feedback_text):
    """
    Use spaCy POS tagging to extract adjectives from the feedback text.
    """
    doc = nlp(feedback_text)
    adjectives = [token.text.capitalize() for token in doc if token.pos_ == "ADJ"]
    return adjectives

In [ ]:
# Handle special phrases (e.g., "not too crowded")
def process_special_phrases(feedback_text, original_tags):
    special_phrases = {
        "not too crowded": "peaceful",
        "not too expensive": "affordable",
        "too crowded": "overpopulated",
        "too expensive": "luxury",
        "too hot": "tropical",
        "not too hot": "tropical",
        "not too cold": "tropical",
        "too cold": "winter",
        "not too far": "near",
        "not too close": "remote",
        "not too long": "short",
        "not too short": "long",
        "not too busy": "peaceful",
        "too busy": "urban",
        "not too loud": "quiet",
        "too loud": "noisy",
        "not too bright": "dim",
        "too bright": "sunny",
        "not too rainy": "dry",
        "too rainy": "wet"
    }

    for phrase, tag in special_phrases.items():
        if phrase in feedback_text.lower():
            if tag not in original_tags:
                original_tags.append(tag)
            feedback_text = feedback_text.lower().replace(phrase, '')

    return feedback_text

In [ ]:


def fallback_to_llm(feedback_text: str, original_tags: list):
    print("Fallback to LLM")
    # Fallback to LLM
    response = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        messages=[
            {
                "role": "user",
                "content": f"""
                Classify the following text as "feedback", "query", or "assertive chat".
                If it's feedback or a query, provide the relevant tags based on the text and the original tags. Do not put any comment. Just tags please. For each relevant adjective that is not present in the original
                text provide one tag in one word that best fits the adjective you don't have to provide its similar or synonymous tags. Be very diligent in providing the tags. Like not too hot can be cold or warm. The tags has
                to be words. no characters please be very mindful regarding this. The tags can also be noun like place name or sight name. You can keep the original tags if you think they are relevant.
                Return only the tags, in a comma-separated list.
                If it's assertive chat, return "null".

                Example 1:
                Input: "Can you show me something cool?"
                Original tags: ["nature", "mountain"]
                Output: "affordable, scenic, cool"

                Example 2:
                Input: "What is the weather like in Switzerland?"
                Original tags: ["travel", "weather"]
                Output: "mountain, Switzerland, adventure"

                Example 3:
                Input: "I don't care about the price, just show me something unique."
                Original tags: ["nature", "adventure"]
                Output: "unique, scenic, exotic"

                Example 4:
                Input: "Great"
                Original tags: ["nature", "mountain"]
                Output: "null"

                Example 5:
                Input: "Thank You"
                Original tags: ["nature", "mountain"]
                Output: "null"


                Input:
                "{feedback_text}"
                Original tags: {original_tags}
                """
            }
        ],
    )

    result = response['choices'][0]['message']['content'].strip()

    if result.lower() != "null":
        tags = [tag.strip().strip('"') for tag in result.split(',')]
        return tags
    else:
        return None

In [ ]:
# Main processing function
def process_feedback(feedback_text: str, original_tags: list):
    """
    Detect feedback/query, then extract relevant tags either from the map or fallback to LLM.
    """
    feedback_text = process_special_phrases(feedback_text, original_tags)  # Handle special phrases

    words = feedback_text.lower().split()

    adjectives = extract_adjectives(feedback_text)

    # Find tags from predefined map
    # tags = [adjective_map.get(word.capitalize(), None) for word in words]
    tags = [adjective_map.get(adj, None) for adj in adjectives]

    tags = list(set(tag for tag in tags if tag))  # Remove duplicates

    if tags:
        updated_tags = [tag for tag in tags if tag not in original_tags]
        original_tags.extend(updated_tags)
        return original_tags
    else:
        tags = fallback_to_llm(feedback_text, original_tags)
        updated_tags = [tag for tag in tags if tag not in original_tags]
        original_tags.extend(updated_tags)
        return original_tags

In [ ]:
if __name__ == "__main__":
    original_tags = ["nature", "mountain"]
    feedback_text = "I don’t care about price, just unique."

    updated_tags = process_feedback(feedback_text, original_tags)
    print(updated_tags)

['nature', 'mountain', 'exotic']
